# Teste comparativo entre ELM, ELM dual com kernel polinomial e ELM dual com kernel RBF

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
from scipy.spatial.distance import cdist
import random
from numba import jit, cuda
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:
import sys
 
file_path = 'output_final.txt'
sys.stdout = open(file_path, "a")

In [3]:
def L0_norm(w, threshold):
    l0_norm = 0
    for wi in w:
        if abs(wi) > threshold:
            l0_norm += 1
    return l0_norm

In [36]:
def compute_margin(K, y, alpha, b, norm_w, model):
    margin = []
    if model == "RP-IMA dual":
        for i in range(y.shape[0]):
            margin.append(y[i]*(np.dot(alpha * y, K[:, i]) + b)/norm_w)
    elif model == "ELM dual" : 
        for i in range(y.shape[0]):
            margin.append(y[i]*(np.dot(alpha, K[:, i]) + b)/norm_w)
    if min(margin) >= 0:
        return min(margin)
    return 0

## Implementação ELM Dual 

In [5]:
def sigmoid(z):
    sigmoid = 1.0/(1.0 + np.exp(-z))
    return sigmoid 

In [6]:
class RP_IMA_dual(BaseEstimator, ClassifierMixin):

    def __init__(self, n_neurons, kernel='rbf', delta_margin=10e-3, gamma='scale', degree=3,
                eta=0.1, max_fmp_updates=100000, IMA_iterations=20,
                regularization_term=0, seed = 0):
        self.n_neurons = n_neurons   # number of hidden neurons
        self.kernel = kernel         # kernel type
        self.gamma = gamma           # kernel param for RBF
        self.degree = degree         # degree of polynomial kernel
        self.delta_margin = delta_margin    # minimum margin increment
        self.eta = eta
        self.max_fmp_updates = max_fmp_updates
        self.IMA_iterations = IMA_iterations
        self.regularization_term = regularization_term # regularization parameter
        self.threshold = 0.1
        self.iterations = 1
        self.seed = seed

    # apply linear kernely * 
    @staticmethod
    @jit(target_backend='cuda')
    def linear_kernel(H):
        return np.dot(H, np.transpose(H))

    # apply plynomial kernel
    @staticmethod
    @jit(target_backend='cuda')
    def polynomial_kernel(H, degree):
        return (np.dot(H, np.transpose(H)))**degree

    @staticmethod
    def silverman_rule(data):
        n = len(data)
        std_data = np.std(data)
        IQR = np.subtract(*np.percentile(data, [75, 25]))
        h = 0.9 * np.min([std_data, IQR/1.34]) * n**(-0.2)
        return h

    @staticmethod
    def rbf_kernel(H, gamma):
        pairwise_dists = cdist(H, H, 'euclidean')
        K = np.exp((-pairwise_dists**2)/(gamma**2))
        return K


    # Dual algorithm to training the fixed geometric mergin perceptron
    @staticmethod
    @jit(target_backend='cuda')
    def dual_FMP_algorithm(K, y, alpha, b, eta, fixed_margin, max_fmp_updates):
        stop = False
        norm_w=0              # norm of w vector
        fmp_epochs=0          # number of fmp epochs
        fmp_norm_updates = 0  # number of updates of the norm of w

        for i in range(K.shape[0]):
            # Cálculo da norma do vetor W:
            norm_w += alpha[i] * y[i] * np.dot(alpha * y , K[:, i])
        norm_w = sqrt(norm_w)

        while True:
            # Checking stop condition
            if stop:
                convergence = 1
                return convergence, fmp_norm_updates, fmp_epochs, norm_w, alpha, b
            elif fmp_norm_updates > max_fmp_updates:
                convergence = 0
                return convergence, fmp_norm_updates, fmp_epochs, norm_w, alpha, b
            else:
                error = False
                # subtract delta from alpha_i values before every new perceptron iteration ,
                # making sure that the property alpha_i >= 0 is respected.
                #for i in range(X.shape[0]):
                #    self.alpha[i] = self.alpha[i] - self.delta
                #    if self.alpha[i] < 0:
                #        self.alpha[i] = 0
                # Loop:
                # Delta_b = 0
                for i in range(K.shape[0]):
                    Delta=0
                    # Checking the margin criterion
                    if((y[i] * np.dot(alpha, y * K[:, i])) <= fixed_margin * norm_w):
                        fmp_norm_updates += 1
                        # alpha update:
                        previous_alpha_y = alpha[i]*y[i]
                        if norm_w != 0:
                            alpha = alpha * (1 - (eta*fixed_margin)/norm_w)
                        alpha[i] += eta
                        Delta = alpha[i]*y[i] - previous_alpha_y
                        # Delta_b += Delta
                        # norm update:
                        norm_w = sqrt(norm_w**2 + Delta * np.dot(alpha, y * K[i, :]))
                        error = True
                        #b += eta * y[i]
                norm_w=0
                for i in range(K.shape[0]):
                    # Cálculo da norma do vetor W:
                    norm_w += alpha[i] * y[i] * np.dot(alpha * y , K[:, i])
                norm_w = sqrt(norm_w)
                if not error:
                    stop = True
                fmp_epochs+=1


    def IM_algorithm(self, X, y):
        #alpha = np.array([random.uniform(0, 1) for i in range(X.shape[0])]) #
        alpha = np.ones((X.shape[0])) * 0.0001
        self.fixed_margin = 0
        t = 0
        convergence = 1
        self.updates = 0
        iterations = 0
        self.b=0  # bias term
        # perceptron kernel matrix
        if self.kernel == 'linear':
            self.K = self.linear_kernel(X) + np.identity(X.shape[0]) * self.regularization_term
        elif self.kernel == 'poly':
            self.K = self.polynomial_kernel(X, self.degree) + np.identity(X.shape[0]) * self.regularization_term
        elif self.kernel == 'rbf':
            self.gamma = self.silverman_rule(X)
            self.K = self.rbf_kernel(X, self.gamma) + np.identity(X.shape[0]) * self.regularization_term
        
        while convergence == 1 and t < self.IMA_iterations:
            gamma1 = []
            gamma2 = []
            #print(f'{self.fixed_margin:1.30f}')
            convergence, updates_, iterations, norm_w, alpha, b = self.dual_FMP_algorithm(self.K, y, alpha, self.b, self.eta, self.fixed_margin, self.max_fmp_updates)
            self.updates += updates_
            if convergence == 1 or t==0:
                self.alpha = deepcopy(alpha)
                self.b = 0#b
                self.norm_w = norm_w
                for i in range(0, y.shape[0]):
                    aux = np.dot(self.alpha * y, self.K[:, i])
                    if y[i] == 1:
                        gamma1.append(y[i]*((aux + self.b)/norm_w))
                    else:
                        gamma2.append(y[i]*((aux + self.b)/norm_w))
                gamma1 = np.array(gamma1)
                gamma2 = np.array(gamma2)
                gamma1 = gamma1[gamma1>=0]
                gamma2 = gamma2[gamma2>=0]
                min_gamma1 = 0
                min_gamma2 = 0
                if len(gamma1) > 0:
                    min_gamma1 = min(gamma1)
                if len(gamma2) > 0:
                    min_gamma2 = min(gamma2)
                self.fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*self.fixed_margin])
                #print(f"IMA: {t}")
            t += 1
        self.iterations = t


    def fit(self, X, y):
        # 1 - Adding polarization term
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        n = X_new.shape[1]
        # 2 - Geração de pesos aleatórios
        random.seed(self.seed)
        self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        # 3 - Aplicação da função de ativação
        self.H = sigmoid(np.dot(X_new, self.Z))
        # 4 - IMA Dual:
        self.y_train = y
        self.IM_algorithm(self.H, y)
        # Cálculo do vetor beta para kernel linear
        if self.kernel == 'linear':
            for i in range(0, y.shape[0]):
                self.beta = np.dot(self.alpha * y, self.H)

    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H_ = sigmoid(np.dot(X_new, self.Z))

        if self.kernel == 'linear':
            y_predicted = np.dot(H_, self.beta)
        elif self.kernel == 'poly':
            y_predicted = np.zeros(X.shape[0])
            for j in range(X.shape[0]):
                for i in range(X.shape[0]):
                    y_predicted[j] += self.alpha[i] * self.y_train[i] * (np.dot(self.H[i], H_[j]) + 1)**self.degree
        elif self.kernel == 'rbf':
            y_predicted = np.zeros(X.shape[0])
            for j in range(H_.shape[0]):
                for i in range(self.H.shape[0]):
                    y_predicted[j] += self.alpha[i] * self.y_train[i] * np.exp((-np.linalg.norm(H_[j] - self.H[i])**2)/self.gamma**2)
                #y_predicted[j] += self.b
        y_predicted[y_predicted>=0] = 1
        y_predicted[y_predicted<0] = -1
        return y_predicted

## Implementação ELM tradicional (forma Dual)

In [27]:
class ELM_dual(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons, kernel='rbf', regularization_term= 0, degree=3, seed = 0):
        self.n_neurons = n_neurons
        self.kernel = kernel
        self.degree = degree
        self.regularization_term = regularization_term
        self.iterations = 1
        self.seed = seed
        
    # apply linear kernel
    @staticmethod
    @jit(target_backend='cuda')
    def linear_kernel(H):
        return np.dot(H, np.transpose(H))

    # apply plynomial kernel
    @staticmethod
    @jit(target_backend='cuda')
    def polynomial_kernel(H, degree):
        return (np.dot(H, np.transpose(H)))**degree

    @staticmethod
    def silverman_rule(data):
        n = len(data)
        std_data = np.std(data)
        IQR = np.subtract(*np.percentile(data, [75, 25]))
        h = 0.9 * np.min([std_data, IQR/1.34]) * n**(-0.2)
        return h

    @staticmethod
    def rbf_kernel(H, gamma):
        pairwise_dists = cdist(H, H, 'euclidean')
        K = np.exp((-pairwise_dists**2)/(gamma**2))
        return K
               
    def fit(self, X, y):
        # 1 - Adding polarization term 
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        n = X_new.shape[1]
        # 2 - Geração de pesos aleatórios
        random.seed(self.seed)
        self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        # 3 - Aplicação da função de ativação
        self.H = sigmoid(np.dot(X_new, self.Z))
        # 4 - Aplicação do kernel
        if self.kernel == 'linear':
            self.K = self.linear_kernel(self.H)
        elif self.kernel == 'poly':
            self.K = self.polynomial_kernel(self.H, self.degree)
        elif self.kernel == 'rbf':
            self.gamma = self.silverman_rule(self.H)
            self.K = self.rbf_kernel(self.H, self.gamma) + np.identity(self.H.shape[0]) * self.regularization_term
        # 5 - Cálculo de alpha:
        self.alpha = np.dot(np.linalg.pinv(self.K, hermitian=True), y)
        # 6 Cálculo do vetor beta
        if self.kernel == 'linear':
            self.beta = np.dot(np.transpose(self.H), self.alpha)
        self.norm_w = 0
        for i in range(self.K.shape[0]):
            # Cálculo da norma do vetor W:
            self.norm_w += self.alpha[i] * np.dot(self.alpha, self.K[:, i])
        self.norm_w = sqrt(self.norm_w)
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H_ = sigmoid(np.dot(X_new, self.Z))
        if self.kernel == 'linear':
            y_predicted = np.dot(H_, self.beta)
        elif self.kernel == 'poly':
            y_predicted = np.zeros(X.shape[0])
            for j in range(H_.shape[0]):
                for i in range(self.H.shape[0]):
                    y_predicted[j] += self.alpha[i] * (np.dot(self.H[i], H_[j]) + 1)**self.degree
        elif self.kernel == 'rbf':
            y_predicted = np.zeros(X.shape[0])
            for j in range(H_.shape[0]):
                for i in range(self.H.shape[0]):
                    y_predicted[j] += self.alpha[i] * np.exp((-np.linalg.norm(H_[j] - self.H[i])**2)/self.gamma**2)
        y_predicted[y_predicted>=0] = 1
        y_predicted[y_predicted<0] = -1
        return y_predicted

## Funções para realizar a testagem dos modelos

In [8]:
def apply_cross_validation(model, X, y, params={}, n_splits=10, seed=1, model_title="Model"):
    k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    accuracy = np.zeros(n_splits)
    train_accuracy = np.zeros(n_splits)
    precision = np.zeros(n_splits)
    recall = np.zeros(n_splits)
    f1 = np.zeros(n_splits)
    iterations = np.zeros(n_splits)
    seed = [10, 22, 52, 59, 1, 21, 99, 85, 74, 69]
    
    norm_L0_1 = np.zeros(n_splits)
    norm_L0_2 = np.zeros(n_splits)
    norm_L0_3 = np.zeros(n_splits)
    norm_L0_4 = np.zeros(n_splits)
    
    margin = np.zeros(n_splits)
    
    idx = 0  
    for train_indices, test_indices in k_fold.split(X, y):
        X_train = X[train_indices]
        X_test = X[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]
        
        # Model apply:
        params['seed'] = seed[idx]
        clf = model(**params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_train = clf.predict(X_train)

        # Calculating accuracy, precision, reall and F1 score:
        precision[idx] = precision_score(y_test, y_pred)
        recall[idx] = recall_score(y_test, y_pred)
        accuracy[idx] = accuracy_score(y_test, y_pred)
        train_accuracy[idx] = accuracy_score(y_train, y_pred_train)
        f1[idx] = f1_score(y_test, y_pred)
        margin[idx] = compute_margin(clf.K, y_train, clf.alpha, 0, clf.norm_w, model_title)
        
        alpha = clf.alpha
        norm = LA.norm(alpha, ord=2)
        alpha = alpha/norm
        norm_L0_1[idx] = L0_norm(alpha, 0.1 * alpha.max())
        norm_L0_2[idx] = L0_norm(alpha, 0.01 * alpha.max())
        norm_L0_3[idx] = L0_norm(alpha, 0.001 * alpha.max())
        norm_L0_4[idx] = L0_norm(alpha, 0.0001 * alpha.max())
        iterations[idx] = clf.iterations
        idx +=1
        
    mean_accuracy = '{:.4f}'.format(accuracy.mean()) + "+/-" + '{:.4f}'.format(accuracy.std())
    mean_accuracy_train = '{:.4f}'.format(train_accuracy.mean()) + "+/-" + '{:.4f}'.format(train_accuracy.std())
    mean_precision ='{:.4f}'.format(precision.mean()) + "+/-" + '{:.4f}'.format(precision.std())
    mean_recall = '{:.4f}'.format(recall.mean()) + "+/-" + '{:.4f}'.format(recall.std())
    mean_f1 = '{:.4f}'.format(f1.mean()) + "+/-" + '{:.4f}'.format(f1.std())
    mean_margin = '{:E}'.format(margin.mean()) + "+/-" + '{:E}'.format(margin.std())
    
    print(f"******************** {model_title} ********************") 
    print("Mean Train Accuracy: " + mean_accuracy_train) 
    print("Mean Accuracy: " + mean_accuracy) 
    print("Mean Precision: " + mean_precision) 
    print("Mean Recall: " + mean_recall) 
    print("Mean F1 Score: " + mean_f1) 
    print("Mean Margin: " + mean_margin) 
    print("Norm L0 (10%): " + '{:.9f}'.format(norm_L0_1.mean())+ "+/-" + '{:.9f}'.format(norm_L0_1.std()))
    print("Norm L0 (1%): " + '{:.9f}'.format(norm_L0_2.mean())+ "+/-" + '{:.9f}'.format(norm_L0_2.std()))
    print("Norm L0 (0.1%): " + '{:.9f}'.format(norm_L0_3.mean())+ "+/-" + '{:.9f}'.format(norm_L0_3.std()))
    print("Norm L0 (0.01%): " + '{:.9f}'.format(norm_L0_4.mean())+ "+/-" + '{:.9f}'.format(norm_L0_4.std()))
    print(f"Iterations: {iterations.mean()}") 
    print('\n\n')
    return mean_accuracy, mean_precision, mean_recall, mean_f1


In [39]:
def run_test(model, X, y, params={}, gridsearch_params={}, model_title="Model"):
    accuracy = list(np.empty(3))
    precision = list(np.empty(3))
    recall = list(np.empty(3))
    f1 = list(np.empty(3))
    N = 1000 if X.shape[0]>=1000 else X.shape[0]
    X = preprocessing.normalize(X, axis=0)
    for idx, n_neurons in enumerate([N, int(N/2), int(N/3)]):
        params['n_neurons'] = n_neurons
        model_params = params
        if gridsearch_params != {}:
            clf = GridSearchCV(model(**params), gridsearch_params , scoring='accuracy', refit=True, cv=10, verbose=0)
            # Run fit with all sets of parameters.
            clf.fit(X, y)
            # Results of Grid Search
            model_params = {**params, **clf.best_params_}
        print(model_params)
        accuracy[idx], precision[idx], recall[idx], f1[idx] = apply_cross_validation(model, X, y, model_params, n_splits=10, model_title=model_title)
    return  accuracy, precision, recall, f1

In [10]:
accuracy_elm = {}
precision_elm = {} 
recall_elm = {}
f1_elm = {}
accuracy_elm_ima = {}
precision_elm_ima = {} 
recall_elm_ima = {}
f1_elm_ima = {}

## Testes:

### Iris 

In [11]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1

dataset = 'IRI' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

/tmp/ipykernel_11126/3352473659.py:57: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  norm_w += alpha[i] * y[i] * np.dot(alpha * y , K[:, i])


### Synthetic Control

In [12]:
synthetic_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))

dataset = 'SYN' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Ionosphere

In [13]:
ionosphere_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())

dataset = 'ION' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Wine

In [14]:
wine_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
y = y.flatten()

dataset = 'WIN' 
print(dataset)
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### WDBC

In [15]:
wdbc_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())

dataset = 'WDBC' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Sonar

In [16]:
sonar_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
dataset = 'SON' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Diabetes

In [17]:
pima_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())

dataset = 'DIA' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Heart

In [41]:
statlog_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())

dataset = 'HEA' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Banknote

In [19]:
# read in banknote authentication set
banknotes = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
y = y.flatten()

dataset = 'BAN' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Mammographic

In [20]:
mammo = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())

dataset = 'MAM' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Haberman

In [21]:
haberman = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())

dataset = 'HAB' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Transfusion

In [22]:
transfusion = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())

dataset = 'TRA' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Australian

In [23]:
australian = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())

dataset = 'AUS' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Breast Cancer

In [24]:
breast = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())

dataset = 'BRE' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Glass

In [25]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1

dataset = 'GLA' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Spam

In [28]:
spam = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/spam/spambase.data', header=None, sep=",", engine='python')
y = spam[57].to_numpy()
X = spam.drop([57], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())

dataset = 'SPA' 
print(dataset) 
params = {'regularization_term': 1} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, params, model_title="ELM dual")

params = {'kernel' : 'rbf', 'regularization_term': 10}
grid_params = {}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Mushroom

In [29]:
df = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())

dataset = 'MUS' 
print(dataset) 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, model_title="ELM dual")

params = {'kernel' : 'rbf', 'regularization_term': 0}
grid_params = {}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

### Robot

In [40]:
robot_dataset = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))

dataset = 'ROB' 
print(dataset) 
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]} 
model = ELM_dual
accuracy_elm[dataset], precision_elm[dataset], recall_elm[dataset], f1_elm[dataset] = run_test(model, X, y, gridsearch_params=grid_params, model_title="ELM dual")

params = {'kernel' : 'rbf'}
grid_params = {'regularization_term':[0, 0.001, 0.01, 1, 10]}
model = RP_IMA_dual
accuracy_elm_ima[dataset], precision_elm_ima[dataset], recall_elm_ima[dataset], f1_elm_ima[dataset] = run_test(model, X, y, params, grid_params, model_title="RP-IMA dual")

## Tabela de Resultados

In [31]:
column_names = pd.DataFrame([["Dual ELM (RBF)", "N"], 
                             ["Dual ELM (RBF)", "N/2"], 
                             ["Dual ELM (RBF)", "N/3"],
                             ["Dual RP-IMA (RBF)", "N"], 
                             ["Dual RP-IMA (RBF)", "N/2"], 
                             ["Dual RP-IMA (RBF)", "N/3"]],
                             columns=["Data sets", ""])

datasets = ['IRI', 'SYN', 'ION', 'BAN', 'WIN', 'WDBC', 'SON', 'DIA', 'HEA', 
            'MAM', 'HAB', 'TRA', 'AUS', 'BRE', 'GLA', 'MUS', 'ROB', 'SPA']

rows=[]
for dataset in datasets:
    rows.append([accuracy_elm[dataset][0], accuracy_elm[dataset][1], accuracy_elm[dataset][2], accuracy_elm_ima[dataset][0], accuracy_elm_ima[dataset][1], accuracy_elm_ima[dataset][2]])

columns = pd.MultiIndex.from_frame(column_names)
df = pd.DataFrame(rows, columns=columns, index=datasets)
print(df)

In [32]:
print(df.to_latex())

/tmp/ipykernel_11126/2350534188.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex())


In [33]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
print(X.shape)

In [38]:
haberman = pd.read_csv('~/Documents/UFMG/Mastering/article/data sets/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())

clf = RP_IMA_dual(n_neurons=100, eta=0.1, regularization_term=0, delta_margin=10^-1, IMA_iterations=200, max_fmp_updates=10000, seed=1)
m = clf.fit(X, y)

print(accuracy_score(y, clf.predict(X)))


clf2 = ELM_dual(100, seed=1, regularization_term=0)
clf2.fit(X, y)

print(accuracy_score(y, clf2.predict(X)))

m = compute_margin(clf.K, y, clf2.alpha, 0, clf2.norm_w, "ELM dual")
print(m)
m = compute_margin(clf.K, y, clf.alpha, 0, clf.norm_w, "RP-IMA dual")
print(m)